### BW38 - 간단한 인터페이스의 경우 클래스 대신 함수를 받아야 한다.

- 파이썬 내장 API 중 상당수는 함수를 전달해서 동작을 원하는 대로 바꿀 수 있게 해준다.
- 전달한 함수를 실행 하는 경우 이런 함수를 훅(hook)이라고 부른다.
<br/> Ex. 이름이 들어 있는 리스트를  정렬하는 예제

In [7]:
names = ['소크라테스', '아르키메데스', '플라톤', '아리스토텔레스']
names.sort(key=len) #  key 훅으로 len 내장 함수를 전달
print(names)

['플라톤', '소크라테스', '아르키메데스', '아리스토텔레스']


- 함수는 클래스보다 정의하거나 기술하기가 더 쉬우므로 훅으로 사용하기에는 함수가 이상적이다.
<br/> → 일급 시민 객체로 취급하기 때문에

In [ ]:
from collections import defaultdict

def log_missing(): # Ex. defaultdict 클래스의 동작을 사용자 정의하고 싶다고 하자.
    print('키 추가됨')
    return 0

log_missing 같은 함수를 넘기면 정해진 동작과 부수효과를 분리할 수 있기 때문에 API를 더 쉽게 만든다.

In [14]:
current = {'초록': 12, '파랑': 3}
increments = [
    ('빨강', 5),
    ('파랑', 17),
    ('주황', 9),
]

result = defaultdict(log_missing, current)
print('이전:', dict(result))

for key, amount in increments:
    result[key] += amount
    
print('이후:', dict(result))

이전: {'초록': 12, '파랑': 3}
키 추가됨
키 추가됨
이후: {'초록': 12, '파랑': 20, '빨강': 5, '주황': 9}


Ex. 기본값 훅을 defaultdict에 넘겨서 찾을 수 없는 키의 총 개수를 센다고 하면

In [3]:
def increment_with_report(current, increments):
    added_count = 0

    def missing():
        nonlocal added_count  # 상태가 있는 클로저
        added_count += 1
        return 0

    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount

    return result, added_count

result, count = increment_with_report(current, increments)
assert count == 2


- defualtdict는 missing훅이 상태를 유지한다는 사실을 모르지만, ~~(?)~~ Q. 상태를 관리한다?
<br/> increment_with_report 함수를 실행하면 튜플의 요소로 기대한 결과 값인 2를 볼수 있다.
<br/> → 인터페이스에서 간단한 함수를 인자로 받으면 클로저 안에 상태를 감추는 기능 계층을 쉽게 추가함

<hr/>

- 상태를 다루기 위한 훅으로 클로저를 사용하면 상태가 없는 함수에 비해 읽기 어렵다.
<br/> → 내가 알고싶은 상태를 저장하는 작은 클래스를 정의 하는게 방법이다.

In [15]:
class CountMissing:
    def __init__(self):
        self.added = 0

    def missing(self):
        self.added += 1
        return 0

- 파이썬에서는 일급시민 객체로 
<br/> CountMissing.missing 메소드를 직접 참조해서 defaultdict의 기본값 훅으로 넘길수 있다.
<br/> → 어떤 함수 인터페이스를 만족하는 객체 인스턴스를 만드는 것은 아주 쉽다.

In [16]:
counter = CountMissing()
result = defaultdict(counter.missing, current) # 메서드 참조
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

- 코드를 처음 보는 사람들은 counter와 missing 두 개 이상의 상태를 파악해야 한다.
<br/> → 이런 경우를 명확하게 하기위해 __ call__ 특별 메서드를 정의해준다.

In [5]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0

    def __call__(self): # 객체를 함수처럼 호출할 수 있게 해준다.
        self.added += 1
        return 0


counter = BetterCountMissing()
assert counter() == 0
assert callable(counter) #  callable 이 이런 인스턴스에 대해서는 True를 반환. / 호출가능객체

- BetterCountMissing 인스턴스를 defaultdict의 디폴트 값 훅으로 사용해서 존재하지 않는 키에 접근한 횟수 추적

In [17]:
counter = BetterCountMissing()
result = defaultdict(counter, current) # __call__에 의존함
for key, amount in increments:
    result[key] += amount
assert counter.added == 2

- __ call__ 내부에서 어떤 일이 벌어 지는에 대해 전혈 알 필요가 없다.

### 기억하자
- 파이썬의 여러 컴포넌트 사이에 간단한 인터페이스가 필요할 떄는 
<br/> 클래스를 정의하고 인스턴스화하는 대신 간단히 함수를 사용할 수 있다.
- 파이썬 함수나 메서드는 일급 시민이다 따라서 함수나 함수 참조를 식에 사용할 수 있다.
- __ call__ 특별 메서드를 사용하면 클래스의 인스턴스인 객체를 일반 파이썬 함수처럼 호출할 수 있다